In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import torch 
from torch import nn 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn import metrics

In [2]:
torch.cuda.is_available()

False

## Problem 3

### Make the NN model

In [3]:
data_path = 'data_assignment_1.csv'

#Data import
df3 = pd.read_csv(data_path)

In [4]:
df3.iloc[:, 3:]

,Frontal_Sup,Frontal_Inf,Cingulum_Ant,Cingulum_Post,Parietal_Sup,Parietal_Inf,Occipital_Sup,Occipital_Inf,Temporal_Sup,Temporal_Inf
0,0.541704,0.553985,0.577727,0.502631,0.539654,0.562739,0.584094,0.656325,0.529551,0.521926
1,0.665915,0.477778,0.525422,0.473192,0.589977,0.656130,0.677115,0.571440,0.430463,0.499023
2,0.385500,0.535842,0.651637,0.556026,0.451074,0.602841,0.578349,0.626247,0.511696,0.531485
3,0.601210,0.500756,0.614601,0.581634,0.489078,0.615674,0.579836,0.548231,0.499937,0.612477
4,0.529871,0.521045,0.576609,0.557342,0.494459,0.577652,0.668027,0.520755,0.521910,0.503761
...,...,...,...,...,...,...,...,...,...,...
95,0.555299,0.597638,0.553309,0.523025,0.525431,0.667517,0.502397,0.601756,0.494372,0.637729
96,0.524961,0.631617,0.382947,0.410978,0.505315,0.611400,0.605452,0.625818,0.445138,0.534949
97,0.490657,0.652892,0.654747,0.656929,0.521088,0.702478,0.640216,0.537376,0.642808,0.598981
98,0.584763,0.602108,0.465685,0.602878,0.693304,0.707188,0.685981,0.683557,0.697648,0.688851


In [5]:
df3.iloc[:, 2]

0     0
1     0
2     0
3     0
4     0
     ..
95    1
96    1
97    1
98    1
99    1
Name: diagnosis, Length: 100, dtype: int64

In [6]:
#Shuffle the data
df3_s = df3.sample(frac = 1)

In [7]:
#k_fold preparation
k = 5
data_lst = []
indices_lst = []

for i in range(k):
    indices_lst.append(int(len(df3_s)/k)*i)
indices_lst.append(len(df3_s))


for j in range(k):
    #Obtained train_test splitted dataframe
    split_test = df3_s[indices_lst[j]:indices_lst[j+1]]
    split_train = df3_s.drop(split_test.index)
    
    train_X = split_train.iloc[:, 3:].values
    train_y = split_train.iloc[:, 2].values
    
    test_X = split_test.iloc[:, 3:].values
    test_y = split_test.iloc[:, 2].values
    
    data_lst.append([[train_X,train_y],[test_X,test_y]])

In [8]:
#Create Model, referenced tutorial
class NNClassifier(nn.Module):
    def __init__(self, input_size = None):
        super().__init__()  
        layers = []
        #input
        layers.append(nn.Linear(input_size, 8))
        layers.append(nn.ReLU())
        #hidden
        layers.append(nn.Linear(8,6))
        layers.append(nn.ReLU())
        #output
        layers.append(nn.Linear(6, 2))

        self.layers = nn.Sequential(*layers)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits
    
    def predict(self, X):
        self.eval()
        with torch.no_grad():
            pred = self(X)
            return int(pred.argmax().detach())     
        

In [9]:
input_len = 10
model = NNClassifier(input_len)
print(model)

NNClassifier(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=6, bias=True)
    (3): ReLU()
    (4): Linear(in_features=6, out_features=2, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (softmax): Softmax(dim=1)
)


In [10]:
# Referenced from tutorial
def train_model(model, train_dl, optimizer, loss_f, nr_epochs, print_loss_every=10):
    for t in range(nr_epochs):
        model.train()
        nr_batches = len(train_dl)
        total_loss = 0
        optimizer.zero_grad()
        
        
        pred_lst = []
        target_lst = []
        
        for _, (X, y) in enumerate(train_dl):
            pred = model(X)
            loss = loss_f(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            pred_lst.append(pred.detach().numpy())
            target_lst.append(y.detach().numpy())
        
        pred_lst = np.concatenate(pred_lst, axis=0)
        target_lst =  np.concatenate(target_lst, axis=0)

        
        if t % print_loss_every == 0:
            print(f"Epoch {t} loss {total_loss / nr_batches}, accuracy {metrics.accuracy_score(target_lst, pred_lst.argmax(1))}")
            
    print("Training complete")
    
def eval_model(model, dl):
    model.eval()
    targets = []
    predictions = []
    for _, (X, y) in enumerate(dl):
        pred = model(X)
        targets += list(y.detach().cpu().numpy())
        predictions += list(pred.argmax(1).detach().cpu().numpy())
        
    accuracy = metrics.accuracy_score(targets, predictions)
    precision = metrics.precision_score(targets,predictions)
    recall = metrics.recall_score(targets,predictions)
    fpr, tpr, _ = metrics.roc_curve(targets, predictions, pos_label=1)
    auc = metrics.auc(fpr,tpr)
    print('Accuracy: '+ str(accuracy))
    print('Precision: '+ str(precision))
    print('Recall: ' + str(recall))
    print('AUC: ' + str(auc))
    
    return [accuracy,precision,recall,auc]

In [11]:
class df2Dataset(Dataset):
    def __init__(self, df_lst, y_dtype=torch.int64):
        self.X = torch.tensor(df_lst[0],dtype=torch.float32)
        self.y = torch.tensor(df_lst[1],dtype=torch.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [12]:
#Train model and evalute
score_lst = []
fold_cnt = 1
no_epoch = 100


for i in data_lst:
    train_dl = DataLoader(df2Dataset(i[0])) 
    test_dl = DataLoader(df2Dataset(i[1]))
    print("Current fold: "+ str(fold_cnt))
    fold_cnt+=1
    
    model = NNClassifier(10) 
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
    loss_function = nn.CrossEntropyLoss()
    
    train_model(model, train_dl, optimizer, loss_function,no_epoch, )
    score_lst.append(eval_model(model,test_dl))
    

Current fold: 1
Epoch 0 loss 0.6978609502315521, accuracy 0.4875
Epoch 10 loss 0.6916820041835308, accuracy 0.525
Epoch 20 loss 0.6843051061034202, accuracy 0.575
Epoch 30 loss 0.6558821562677621, accuracy 0.6125
Epoch 40 loss 0.6208465065807104, accuracy 0.65
Epoch 50 loss 0.5996514555066824, accuracy 0.65
Epoch 60 loss 0.5916382543742656, accuracy 0.65
Epoch 70 loss 0.578793865069747, accuracy 0.675
Epoch 80 loss 0.5759369240142405, accuracy 0.6625
Epoch 90 loss 0.5687971205450595, accuracy 0.6625
Training complete
Accuracy: 0.5
Precision: 0.5333333333333333
Recall: 0.7272727272727273
AUC: 0.4747474747474747
Current fold: 2
Epoch 0 loss 0.6954662814736366, accuracy 0.525
Epoch 10 loss 0.6933379180729389, accuracy 0.525
Epoch 20 loss 0.69315440133214, accuracy 0.525
Epoch 30 loss 0.6926877319812774, accuracy 0.525
Epoch 40 loss 0.691290645301342, accuracy 0.525
Epoch 50 loss 0.6853299655020237, accuracy 0.525
Epoch 60 loss 0.6690097652375698, accuracy 0.6
Epoch 70 loss 0.6443586748093

### Log Regression and SVM

In [13]:
data_lst[0][1][1]

array([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [14]:
svm_score_lst = []
log_score_lst = []

fold_cnt = 1
for i in data_lst:
    print("Current fold: "+ str(fold_cnt))
    fold_cnt+=1
    
    X_train = i[0][0]
    Y_train = i[0][1]
    X_test = i[1][0]
    Y_test = i[1][1]
    
    
    log_model = LogisticRegression(random_state=0, max_iter=200)
    log_model.fit(X_train, Y_train)

    logtrain_pred = log_model.predict(X_train)
    logtest_pred = log_model.predict(X_test)
    
    
    accuracy = metrics.accuracy_score(Y_test,logtest_pred)
    precision = metrics.precision_score(Y_test,logtest_pred)
    recall = metrics.recall_score(Y_test,logtest_pred)
    fpr, tpr, _ = metrics.roc_curve(Y_test,logtest_pred, pos_label=1)
    auc = metrics.auc(fpr,tpr)
    print("log test accuracy: ", accuracy)

    log_score_lst.append([accuracy,precision,recall,auc])
    
    svm = SVC(gamma='auto', kernel='linear')
    svm.fit(X_train, Y_train)
    svmtrain_pred = svm.predict(X_train)
    svmtest_pred = svm.predict(X_test)
    
    accuracy = metrics.accuracy_score(Y_test,svmtest_pred)
    precision = metrics.precision_score(Y_test,svmtest_pred)
    recall = metrics.recall_score(Y_test,svmtest_pred)
    fpr, tpr, _ = metrics.roc_curve(Y_test,svmtest_pred, pos_label=1)
    auc = metrics.auc(fpr,tpr)
    print("svm test accuracy: ", accuracy)
    
    log_score_lst.append([accuracy,precision,recall,auc])
    

Current fold: 1
log test accuracy:  0.6
svm test accuracy:  0.6
Current fold: 2
log test accuracy:  0.6
svm test accuracy:  0.4
Current fold: 3
log test accuracy:  0.65
svm test accuracy:  0.6
Current fold: 4
log test accuracy:  0.6
svm test accuracy:  0.5
Current fold: 5
log test accuracy:  0.65
svm test accuracy:  0.7


# Problem 4

In [15]:
import os
import cv2
from torch.utils.data import Dataset
from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torchvision.transforms import CenterCrop
from torch.nn import functional as F
import torch

from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import time
import os

In [16]:
BATCH_SIZE = 16

### Data input and train_test_split

In [17]:
data_path_4 = '/Users/charliejiang/Documents/Stanford/Machine-Learning-for-Neuroimaging/HW2/data_assignment_2/lgg-mri-segmentation/kaggle_3m'

metadata = pd.read_csv(data_path_4 + '/data.csv')
patient_id = metadata['Patient']

In [18]:
folder_lst = []
for i in os.listdir(data_path_4):
    if i[0:12] in patient_id.values:
        folder_lst.append(os.path.join(data_path_4,i))
print(len(folder_lst))

110


In [19]:
img_lst = []
mask_lst = []
#Obtain paths for both mask and MRI images at same time, ensuring path order is preservered
for i in folder_lst:
    file_lst = os.listdir(i)
    for j in file_lst:
        if j[-8:] != 'mask.tif':
            img_lst.append(os.path.join(data_path_4,i,j))
            mask_lst.append(os.path.join(data_path_4,i,j[:-4])+'_mask.tif')            

In [20]:
#Test if input data is standarized
shape_0 =  cv2.imread(img_lst[0]).shape
for i in img_lst:    
    if cv2.imread(i).shape != shape_0:
        print("This aint it")
        break
mshape_0 =  cv2.imread(img_lst[0]).shape    
if mshape_0 != shape_0:
    print("Nah")
for i in mask_lst:    
    if cv2.imread(i).shape != mshape_0:
        print("This aint it")
        break


In [21]:
initial_shape = (shape_0[0],shape_0[1])

In [22]:
train_val_imgs,test_imgs,train_val_masks,test_masks = train_test_split(img_lst,mask_lst,test_size=0.2, random_state=42)
train_imgs,val_imgs,train_masks,val_masks = train_test_split(train_val_imgs,train_val_masks,test_size=0.125, random_state=42)

In [23]:
#Verify split is correct

print(len(train_imgs))
print(len(val_imgs))
print(len(test_imgs))

for i in val_imgs:
    if i in train_imgs or i in test_imgs:
        print('Error')
        break
for j in val_masks:
    if j in train_masks or j in test_masks:
        print('Error')
        break
        
for i in test_imgs:
    if i in train_imgs:
        print('Error')
        break
for j in test_masks:
    if j in train_masks:
        print('Error')
        break
        

2750
393
786


In [24]:
class SegmentationDataset(Dataset):
    def __init__(self, imagePaths, maskPaths):
        self.imagePaths = imagePaths
        self.maskPaths = maskPaths
        self.transforms = transforms
    def __len__(self):
        return len(self.imagePaths)
    def __getitem__(self, idx):
        imagePath = self.imagePaths[idx]
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.maskPaths[idx], 0)
        
        image = PIL.to
        image = torch.tensor(image)
        mask = torch.tensor(mask)
        
        return (image, mask)

In [25]:
trainDS = SegmentationDataset(imagePaths=train_imgs, maskPaths=train_masks)
valDS = SegmentationDataset(imagePaths=val_imgs, maskPaths=val_masks)
testDS = SegmentationDataset(imagePaths=test_imgs, maskPaths=test_masks)
print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(valDS)} examples in the val set...")
print(f"[INFO] found {len(testDS)} examples in the test set...")

trainLoader = DataLoader(trainDS, shuffle=True,batch_size = BATCH_SIZE,
    num_workers=os.cpu_count())

valLoader = DataLoader(valDS, shuffle=False,batch_size = BATCH_SIZE,
    num_workers=os.cpu_count())

testLoader = DataLoader(testDS, shuffle=False,batch_size = BATCH_SIZE,
    num_workers=os.cpu_count())


[INFO] found 2750 examples in the training set...
[INFO] found 393 examples in the val set...
[INFO] found 786 examples in the test set...


In [ ]:
trainDS.__getitem__(0)

In [ ]:
class DConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

In [ ]:
class Encoder(Module):
    def __init__(self, channels=(3, 16, 32, 64)):
        super().__init__()
        self.encBlocks = ModuleList(
            [DConvBlock(channels[i], channels[i + 1])
                 for i in range(len(channels) - 1)])
        self.pool = MaxPool2d(2)
    def forward(self, x):
        blockOutputs = []
        for block in self.encBlocks:
            x = block(x)
            blockOutputs.append(x)
            x = self.pool(x)
        return blockOutputs

In [ ]:
class Decoder(Module):
    def __init__(self, channels=(64, 32, 16)):
        super().__init__()
        self.channels = channels
        self.upconvs = ModuleList(
            [ConvTranspose2d(channels[i], channels[i + 1], 2, 2)
                 for i in range(len(channels) - 1)])
        self.dec_blocks = ModuleList(
            [DConvBlock(channels[i], channels[i + 1])
                for i in range(len(channels) - 1)])
    def forward(self, x, encFeatures):
        for i in range(len(self.channels) - 1):
            x = self.upconvs[i](x)
            encFeat = self.crop(encFeatures[i], x)
            x = torch.cat([x, encFeat], dim=1)
            x = self.dec_blocks[i](x)
        return x
    def crop(self, encFeatures, x):
        (_, _, H, W) = x.shape
        encFeatures = CenterCrop([H, W])(encFeatures)
        return encFeatures


In [ ]:
class UNet(Module):
    def __init__(self, encChannels=(3, 16, 32, 64),
                 decChannels=(64, 32, 16),
                 nbClasses=1, retainDim=True,
                 outSize=initial_shape):
        super().__init__()

        self.encoder = Encoder(encChannels)
        self.decoder = Decoder(decChannels)

        self.head = Conv2d(decChannels[-1], nbClasses, 1)
        self.retainDim = retainDim
        self.outSize = outSize
    def forward(self, x):
        encFeatures = self.encoder(x)
        decFeatures = self.decoder(encFeatures[::-1][0],encFeatures[::-1][1:])
        map = self.head(decFeatures)
        if self.retainDim:
            map = F.interpolate(map, self.outSize)
        return map

In [ ]:
u_net=UNet()
print(u_net)

In [ ]:
lr = 0.001
lossfun = BCEWithLogitsLoss()
optimizer = Adam(u_net.parameters(), lr)

In [ ]:
def jaccard_coeff(y_pred, y_true):
        inter = np.dot(y_pred, y_true)
        union = np.sum(y_pred) + np.sum(y_true) - inter
        jaccard_index = inter/union

        return jaccard_index.cpu().numpy()

In [ ]:
def dice_coeff_binary(y_pred, y_true):
        eps = 0.0001
        inter = np.dot(y_pred, y_true)
        union = np.sum(y_pred) + np.sum(y_true) - inter
        return ((2 * inter.float() + eps) / (union.float() + eps)).cpu().numpy()

In [ ]:
# loop over epochs
def train_unet(unet,trainLoader,valLoader, opt,lossFunc,epoch_cnt = 100):
    # Training model
    startTime = time.time()
    
    train_loss = []
    valid_loss = []
    train_accuracy = []
    valid_accuracy = []
    train_dice = []
    valid_dice = []
    
    for e in range(epoch_cnt):
        unet.train()
        
        pred_lst = []
        target_lst = []
        pred_val_lst = []
        train_epoch_loss = []

        for (i, (x, y)) in enumerate(trainLoader):
            pred = unet(x)
            loss = lossFunc(pred, y)
            
            train_epoch_loss.append(loss)

            opt.zero_grad()
            loss.backward()
            opt.step()

            
            pred_lst.append(pred.detach().numpy())
            target_lst.append(y.detach().numpy())
        
        pred_lst = np.concatenate(pred_lst, axis=0)
        target_lst =  np.concatenate(target_lst, axis=0)
        

        with torch.no_grad():
            unet.eval()
            for (x, y) in valLoader:
                pred = unet(x)
                pred_val_lst.append(pred.detach().numpy())
                val_epoch_loss.append(loss(pred,y))
            
        pred_val_lst = np.concatenate(pred_lst, axis=0)
                                    
        avgTrainLoss = train_epoch_loss.mean()
        avgTrainAccuracy = metrics.accuracy_score(target_lst, pred_lst.argmax(1))
        
        avgValLoss = val_epoch_loss.mean()
    
        train_accuracy.append(avgTrainAccuracy)
        val_accuracy.append(avgValAccuracy)
        train_loss.append(avgTrainLoss)
        val_loss.append(avgValLoss)
        
        
        print(f"Epoch {e}: Training Accuracy: {avgTrainAccuracy} Loss: {avgTrainLoss}")
        print(f"Epoch {e}: Validation Accuracy: {avgTrainAccuracy} Loss: {avgTrainLoss}")

    endTime = time.time()
    print("[INFO] total time taken to train the model: {:.2f}s".format(
        endTime - startTime))
    
    #Validation

In [ ]:
train_unet(u_net,trainLoader,valLoader,optimizer,lossfun,epoch_cnt = 100)